In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as web
from datetime import datetime
from pandas_datareader._utils import RemoteDataError
import requests
import json
import yfinance as yf

In [ ]:
data=pd.read_csv("C:\\Users\\Kalpa\\Documents\\PythonFiles\\Stocklist.csv")
#data=pd.read_csv("C:\\Users\\Kalpa\\Documents\\PythonFiles\\finalcountdown.csv")
#data=pd.read_csv("C:\\Users\\Kalpa\\Documents\\PythonFiles\\Megacap.csv")
data.head()

In [ ]:
# Pull stock data from Yahoo finance 

for i in range (len(data)):
 
    stock = web.DataReader(data.loc[i,"Symbol"],data_source='yahoo',start='13/4/2015', end='28/4/2020')
    detail=yf.Ticker(data.loc[i,"Symbol"])
    stock['Log_Ret'] = np.log(stock['Close'] / stock['Close'].shift(1))
    stock['Volatility1yr']=stock['Log_Ret'].rolling(252).std()
    stock['Volatility3yr']=stock['Log_Ret'].rolling(756).std()
    stock['1yrAvgRet']=stock['Log_Ret'].rolling(252).mean()
    stock['3yrAvgRet']=stock['Log_Ret'].rolling(756).mean()
    #Dividend Data
    df=detail.get_dividends().to_frame()
    #populate data
    data.loc[i,"1yrAvgRet"]=stock['1yrAvgRet'][-1]
    data.loc[i,"3yrAvgRet"]=stock['3yrAvgRet'][-1]
    data.loc[i,"1yrDivRet"]=(1/252)*(df[df.index>"2019"].sum()/stock['Close'][-252 if len(stock['Close'])>252 else -len(stock['Close'])])[0]
    data.loc[i,"3yrDivRet"]=(1/756)*(df[df.index>"2017"].sum()/stock['Close'][-756 if len(stock['Close'])>756 else -len(stock['Close'])])[0]
    data.loc[i,"1yrSharpe"]=((stock['1yrAvgRet'][-1]+data.loc[i,"1yrDivRet"])/stock['Volatility1yr'][-1])*np.sqrt(252)
    data.loc[i,"3yrSharpe"]=((stock['3yrAvgRet'][-1]+data.loc[i,"3yrDivRet"])/stock['Volatility3yr'][-1])*np.sqrt(252)
    #format in percentage and annualize
    data.loc[i,"1yrAvgRet"]="{:.2f}%".format(data.loc[i,"1yrAvgRet"]*100*252)
    data.loc[i,"3yrAvgRet"]="{:.2f}%".format(data.loc[i,"3yrAvgRet"]*100*252)
    data.loc[i,"1yrDivRet"]="{:.2f}%".format(data.loc[i,"1yrDivRet"]*100*252)
    data.loc[i,"3yrDivRet"]="{:.2f}%".format(data.loc[i,"3yrDivRet"]*100*252)

In [ ]:
# find top 30 performers and take intersection. Show only where 1yr avg return> 3yr Avg return
k1=pd.merge(data.nlargest(30,'1yrSharpe'),data.nlargest(30,'3yrSharpe')['Symbol'],on='Symbol')
k2=data[data['1yrAvgRet']>data['3yrAvgRet']]
pd.merge(k1,k2['Symbol'],on='Symbol')